In [4]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [5]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [56]:
splitRdd = sc.textFile("file:///home/talentum/test-jupyter/P2/M2/Pair-RDD/selfishgiant.txt"
                      ).flatMap(lambda line: line.split(" "))

print(splitRdd.take(5))

['EVERY', 'afternoon,', 'as', 'they', 'were']


In [9]:
mappedRdd = splitRdd.map(lambda word: (word, 1))
print(mappedRdd.take(5))

[('EVERY', 1), ('afternoon,', 1), ('as', 1), ('they', 1), ('were', 1)]


In [11]:
months = ("Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul")

monthsRdd = sc.parallelize(months)

monthsIndexed0Rdd = monthsRdd.zipWithIndex()

print(monthsIndexed0Rdd.collect())

[('Jan', 0), ('Feb', 1), ('Mar', 2), ('Apr', 3), ('May', 4), ('Jun', 5), ('Jul', 6)]


In [16]:
monthsIndexed1Rdd = monthsIndexed0Rdd.map(lambda x: (x[0], x[1]+1))
print(monthsIndexed1Rdd.collect())

[('Jan', 1), ('Feb', 2), ('Mar', 3), ('Apr', 4), ('May', 5), ('Jun', 6), ('Jul', 7)]


In [17]:
monthsIndexed2Rdd = monthsIndexed0Rdd.mapValues(lambda y: y+1)
print(monthsIndexed2Rdd.collect())

[('Jan', 1), ('Feb', 2), ('Mar', 3), ('Apr', 4), ('May', 5), ('Jun', 6), ('Jul', 7)]


In [19]:
quarters = (1, 1, 1, 2, 2, 2, 3)

quartersRdd = sc.parallelize(quarters)
monthsZipQuarters = monthsRdd.zip(quartersRdd)
print(monthsZipQuarters.collect())

[('Jan', 1), ('Feb', 1), ('Mar', 1), ('Apr', 2), ('May', 2), ('Jun', 2), ('Jul', 3)]


In [21]:
print(monthsZipQuarters.keys().collect())

print(monthsZipQuarters.values().collect())

print(monthsZipQuarters.sortByKey().collect())

['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul']
[1, 1, 1, 2, 2, 2, 3]
[('Apr', 2), ('Feb', 1), ('Jan', 1), ('Jul', 3), ('Jun', 2), ('Mar', 1), ('May', 2)]


In [22]:
reducedByKeyRdd = mappedRdd.reduceByKey(lambda x,y: x+y)
print(reducedByKeyRdd.take(5))

[('EVERY', 1), ('afternoon,', 3), ('as', 9), ('they', 17), ('were', 21)]


In [25]:
flippedRdd = reducedByKeyRdd.map(lambda x: (x[1], x[0]))
print(flippedRdd.take(5))

[(1, 'EVERY'), (3, 'afternoon,'), (9, 'as'), (17, 'they'), (21, 'were')]


In [26]:
orderedRdd = flippedRdd.sortByKey(ascending=False)
print(orderedRdd.take(5))

[(148, 'the'), (85, 'and'), (44, 'he'), (38, 'to'), (33, '')]


In [27]:
# 2nd task

In [28]:
carrierRdd = sc.textFile("file:///home/talentum/test-jupyter/P2/M2/Pair-RDD/flights.csv"
                      ).map(lambda val: val.split(',')).map(lambda column: (column[5], 1))

print(carrierRdd.take(5))

[('WN', 1), ('WN', 1), ('WN', 1), ('WN', 1), ('WN', 1)]


In [34]:
carriersSorted = carrierRdd.reduceByKey(lambda x, y: x + y
                                        ).map(lambda x: (x[1], x[0])
                                        ).sortByKey(ascending = False)

print(carriersSorted.take(3))

[(356167, 'WN'), (175969, 'AA'), (166445, 'OO')]


In [35]:
# 3rd task

In [36]:
cityRdd = sc.textFile("file:///home/talentum/test-jupyter/P2/M2/Pair-RDD/airports.csv"
                     ).map(lambda val: val.split(",")
                          ).map(lambda column: (column[0], column[2]))

print(cityRdd.take(5))

[('iata', 'city'), ('00M', 'BaySprings'), ('00R', 'Livingston'), ('00V', 'ColoradoSprings'), ('01G', 'Perry')]


In [38]:
flightOrigDestRdd = sc.textFile("file:///home/talentum/test-jupyter/P2/M2/Pair-RDD/flights.csv"
                               ).map(lambda val: val.split(',')
).map(lambda column: (column[12], column[13]))

print(flightOrigDestRdd.take(5))

[('IAD', 'TPA'), ('IND', 'BWI'), ('IND', 'JAX'), ('IND', 'LAS'), ('IND', 'PHX')]


In [39]:
origJoinRdd = flightOrigDestRdd.join(cityRdd)
print(origJoinRdd.take(5))

[('OAK', ('BOI', 'Oakland')), ('OAK', ('BUR', 'Oakland')), ('OAK', ('BUR', 'Oakland')), ('OAK', ('BUR', 'Oakland')), ('OAK', ('DEN', 'Oakland'))]


In [40]:
destOrigJoinRdd = origJoinRdd.values().join(cityRdd)
print(destOrigJoinRdd.take(5))

[('ONT', ('Oakland', 'Ontario')), ('ONT', ('Oakland', 'Ontario')), ('ONT', ('Oakland', 'Ontario')), ('ONT', ('Oakland', 'Ontario')), ('ONT', ('Oakland', 'Ontario'))]


In [41]:
citiesCleanedRdd = destOrigJoinRdd.values()
print(citiesCleanedRdd.take(5))

[('Oakland', 'Ontario'), ('Oakland', 'Ontario'), ('Oakland', 'Ontario'), ('Oakland', 'Ontario'), ('Oakland', 'Ontario')]


In [42]:
citiesKV = citiesCleanedRdd.map(lambda cities: (cities, 1))
print(citiesKV.take(5))

[(('Oakland', 'Ontario'), 1), (('Oakland', 'Ontario'), 1), (('Oakland', 'Ontario'), 1), (('Oakland', 'Ontario'), 1), (('Oakland', 'Ontario'), 1)]


In [47]:
citiesReducedSortedRdd = citiesKV.reduceByKey(lambda x,y: x+y).map(lambda x: (x[1],x[0])
                                                                  ).sortByKey(ascending=False)
print(citiesReducedSortedRdd.take(3))

[(5540, ('NewYork', 'Boston')), (5478, ('Boston', 'NewYork')), (4103, ('Chicago', 'NewYork'))]


In [49]:
file3 = "file:///home/talentum/test-jupyter/P2/M2/Pair-RDD/flights.csv"

delayRDD = sc.textFile(file3
                      ).map(lambda x: x.split(",")
                      ).filter(lambda x: int(x[11]) > 15
                      ).map(lambda x: (x[5], x[11]))

print(delayRDD.take(5))

[('WN', '25'), ('WN', '67'), ('WN', '87'), ('WN', '29'), ('WN', '82')]


In [51]:
delayMaxRDD = delayRDD.reduceByKey(lambda x, y: max(int(x), int(y)))

print(delayMaxRDD.take(5))

[('XE', 781), ('YV', 526), ('OH', 680), ('OO', 767), ('UA', 1268)]


In [52]:
file4 = "file:///home/talentum/test-jupyter/P2/M2/Pair-RDD/plane-data.csv"

planeDataRDD = sc.textFile(file4)
print(planeDataRDD.count())

5030


In [54]:
cleanedPlaneDataRDD = planeDataRDD.map(lambda x: x.split(",")
                                 ).filter(lambda x: len(x) == 9
                                 )
cleanedPlaneDataRDD.count()

4481